In [1]:
# half the counts
!tree ../input/deepglobe-land-cover-classification-dataset --filelimit 5

../input/deepglobe-land-cover-classification-dataset
├── class_dict.csv
├── metadata.csv
├── test [172 entries exceeds filelimit, not opening dir]
├── train [1606 entries exceeds filelimit, not opening dir]
└── valid [171 entries exceeds filelimit, not opening dir]

3 directories, 2 files


In [2]:
import pandas as pd
labelmap = pd.read_csv("../input/deepglobe-land-cover-classification-dataset/class_dict.csv")
labelmap

,name,r,g,b
0,urban_land,0,255,255
1,agriculture_land,255,255,0
2,rangeland,255,0,255
3,forest_land,0,255,0
4,water,0,0,255
5,barren_land,255,255,255
6,unknown,0,0,0


In [3]:
# !rm -rf {OUTPUT_DIR}

In [4]:
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import random
import numpy as np
import cv2
from tqdm import tqdm
np.random.seed(42)
random.seed(42)

OUTPUT_DIR = Path('./pix2pixHD-deepglobe-synthesis')
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
                  
TRAIN_DIR = Path('../input/deepglobe-land-cover-classification-dataset/train')

labels = sorted(list(TRAIN_DIR.glob("*_mask.png")))
images = sorted(list(TRAIN_DIR.glob("*_sat.jpg")))

idxs = np.arange(len(labels))
np.random.shuffle(idxs)
labels = np.array(labels)[idxs] 
images = np.array(images)[idxs]
for i, (lp, ip) in tqdm(enumerate(zip(labels, images)), total=len(labels)):
    split = 'train'
    if i<30:
        split = 'test'
        
    img_dir = OUTPUT_DIR / f'{split}_img'
    lab_dir = OUTPUT_DIR / f'{split}_label'
    # print(split,img_dir)
    
    img_dir.mkdir(exist_ok=True, parents=True)
    lab_dir.mkdir(exist_ok=True, parents=True)

    img = cv2.imread(str(ip))
    lab = cv2.imread(str(lp))

    # plt.imshow(img[:,:,::-1])
    # plt.title(ip.name)
    # plt.show()
            
    c0 = (lab == [0,0,0]).all(-1)       # unk
    c1 = (lab == [0,255,255]).all(-1)   # urbanland
    c2 = (lab == [255,255,0]).all(-1)   # agriculture_land 	255 	255 	0
    c3 = (lab == [255,0,255]).all(-1)   # rangeland 	255 	0 	255
    c4 = (lab == [0,255,0]).all(-1)     # forest_land 	0 	255 	0
    c5 = (lab == [0,0,255]).all(-1)     # water 	0 	0 	255
    c6 = (lab == [255,255,255]).all(-1) # barren_land 	255 	255 	255
    
    msk = np.zeros(lab.shape[:2])
    msk[c0]=0
    msk[c1]=1
    msk[c2]=2
    msk[c3]=3
    msk[c4]=4
    msk[c5]=5
    msk[c6]=6
    
    cv2.imwrite(str(img_dir/ip.name), img)
    cv2.imwrite(str(lab_dir/lp.name), msk)

100%|██████████| 803/803 [21:47<00:00,  1.63s/it]


In [5]:
!tree {OUTPUT_DIR} --filelimit 4

pix2pixHD-deepglobe-synthesis
├── test_img [30 entries exceeds filelimit, not opening dir]
├── test_label [30 entries exceeds filelimit, not opening dir]
├── train_img [773 entries exceeds filelimit, not opening dir]
└── train_label [773 entries exceeds filelimit, not opening dir]

4 directories, 0 files


In [6]:
!zip -r ./pix2pixHD-deepglobe-synthesis.zip ./pix2pixHD-deepglobe-synthesis

  adding: pix2pixHD-deepglobe-synthesis/ (stored 0%)
  adding: pix2pixHD-deepglobe-synthesis/train_img/ (stored 0%)
  adding: pix2pixHD-deepglobe-synthesis/train_img/358314_sat.jpg (deflated 0%)
  adding: pix2pixHD-deepglobe-synthesis/train_img/741105_sat.jpg (deflated 0%)
  adding: pix2pixHD-deepglobe-synthesis/train_img/435277_sat.jpg (deflated 0%)
  adding: pix2pixHD-deepglobe-synthesis/train_img/508571_sat.jpg (deflated 0%)
  adding: pix2pixHD-deepglobe-synthesis/train_img/825592_sat.jpg (deflated 0%)
  adding: pix2pixHD-deepglobe-synthesis/train_img/382428_sat.jpg (deflated 0%)
  adding: pix2pixHD-deepglobe-synthesis/train_img/587968_sat.jpg (deflated 0%)
  adding: pix2pixHD-deepglobe-synthesis/train_img/129298_sat.jpg (deflated 0%)
  adding: pix2pixHD-deepglobe-synthesis/train_img/495876_sat.jpg (deflated 0%)
  adding: pix2pixHD-deepglobe-synthesis/train_img/772130_sat.jpg (deflated 0%)
  adding: pix2pixHD-deepglobe-synthesis/train_img/893651_sat.jpg (deflated 0%)
  adding: pix2p